In [91]:
#!pip install yfinance
#!pip install requests
#!pip install beautifulsoup4

In [4]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

#We only care about data from the last two months
end_date = datetime.now()
start_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
end_date_formatted = str(end_date).split(" ")[0]
start_date_formatted = str(start_date).split(" ")[0]

data = yf.download("BMW.DE", start=start_date_formatted, end=end_date_formatted)

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-11,72.919998,75.459999,72.000000,72.419998,72.419998,2873113
2022-03-14,74.639999,76.370003,73.980003,74.430000,74.430000,2514991
2022-03-15,73.629997,75.779999,71.699997,75.320000,75.320000,2601157
2022-03-16,77.540001,79.279999,76.180000,78.400002,78.400002,3188590
2022-03-17,79.339996,79.400002,75.510002,76.669998,76.669998,2207957
2022-03-18,76.199997,76.660004,74.750000,75.900002,75.900002,4736344
2022-03-21,76.239998,77.970001,75.739998,77.440002,77.440002,1686583
2022-03-22,78.349998,79.760002,77.349998,79.059998,79.059998,1732509
2022-03-23,80.339996,80.800003,77.849998,78.260002,78.260002,1942435


In [76]:
#Construct an array of only the data we care about (yesterday, 1wk ago, 2wk ago, 3wk ago, etc up to 6wk)
target_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
target_date_formatted = str(target_date).split(" ")[0]

data.loc[target_date_formatted]

#An array to hold our closing values (from oldest to newest)
closing_values = []

while target_date <= end_date:
    target_row = data.loc[target_date_formatted]
    closing_values.append(target_row['Close'])
    target_date = target_date + relativedelta(weeks=1)
    target_date_formatted = str(target_date).split(" ")[0]
    
closing_values

[89.66000366210938,
 95.30000305175781,
 95.37999725341797,
 88.0199966430664,
 73.0999984741211,
 72.41999816894531,
 75.9000015258789]

In [90]:
#Okay.  We've proven we can get the stock prices.  Now let's prove we can grab the sentences for our sentiment score.
import requests
from bs4 import BeautifulSoup

URL = "https://finance.yahoo.com/quote/TSLA"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
results

news_elements = results.find_all("li", class_="js-stream-content Pos(r)")
for news_element in news_elements:
    title_element = news_element.find("h3")
    subtitle_element = news_element.find("p")
    print(title_element.text)
    print(subtitle_element.text)

Electric vehicles: ‘We’ve seen a fairly steep rise’ in searches, analyst says
Jessica Caldwell, executive director of insights at Edmunds, explains how more consumers are turning to electric vehicles.
Elon Musk Has Billions, but Still Took Out Mortgages. Here's Why
Elon Musk has founded Tesla, SpaceX, and a number of other companies.  In fact, Musk reportedly took out a combined total of $61 million in mortgages in 2018 for several properties in Bel Air and the Bay Area.  This included $50 million in new mortgage debt, as well as a refinance loan that he used to repay a mortgage that he had used to purchase a property in 2017.
Dow Jones Futures: 2022's First Real Market Rally Gains Steam; Five Stocks Flash Buy Signals
The market rally continues to gain steam, with Nvidia among stocks flashing buy signals. Time to go shopping, but don't go on a buying frenzy.
Tesla Stock Vs. BYD Stock: Tesla Berlin Nears Milestone As EV Rival BYD Sets Huge Target
Tesla stock vs. BYD stock: China's BYD i

In [3]:
#There you have it!  We've proven we can get all the data we need to run this network.  All that's left is to run the sentiments
#through a network trained on https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis and add the sentiment score,
#then construct the final dataframe and train a NN on it!
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup

df = pd.read_csv("Largest automakers by market capitalization.csv")

df = df.drop(columns=['price (USD)','Rank'])
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols
df = df[cols]

sentiments = []
six_weeks_ago = []
five_weeks_ago = []
four_weeks_ago = []
three_weeks_ago = []
two_weeks_ago = []
one_week_ago = []
yesterday = []

#We only care about data from the last two months
end_date = datetime.now()
start_date = end_date - relativedelta(weeks=6) - relativedelta(days=1)
end_date_formatted = str(end_date).split(" ")[0]
start_date_formatted = str(start_date).split(" ")[0]

for symbol in df['Symbol'].values:
    print(symbol)

    stock_url = "https://finance.yahoo.com/quote/" + symbol

    page = requests.get(stock_url)

    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
    
    if results is None:
        results = soup.find(id="quoteNewsStream-0-Stream-Proxy")
        sentiments.append("No data!")
    else:
        sentiment_list = []

        news_elements = results.find_all("li", class_="js-stream-content Pos(r)")
        for news_element in news_elements:
            title_element = news_element.find("h3")
            subtitle_element = news_element.find("p")
            sentiment_list.append(title_element.text)
            sentiment_list.append(subtitle_element.text)
        
        sentiments.append(sentiment_list)

    data = yf.download(symbol, start=start_date_formatted, end=end_date_formatted)
    target_date = start_date
    target_date_formatted = str(target_date).split(" ")[0]

    #An array to hold our closing values (from oldest to newest)
    closing_values = []
    
    #print(data)
    #break
        
    while target_date <= end_date:
            try:
                data.loc[target_date_formatted]
            except:
                print(f"Error on symbol {symbol} for {target_date_formatted}")
                closing_values.append('error')
                
            else:
                target_row = data.loc[target_date_formatted]
                closing_values.append(target_row['Close'])
            target_date = target_date + relativedelta(weeks=1)
            target_date_formatted = str(target_date).split(" ")[0]
            
    six_weeks_ago.append(closing_values[0])
    five_weeks_ago.append(closing_values[1])
    four_weeks_ago.append(closing_values[2])
    three_weeks_ago.append(closing_values[3])
    two_weeks_ago.append(closing_values[4])
    one_week_ago.append(closing_values[5])
    yesterday.append(closing_values[6])

df['Stock Price (6wk ago),'] = six_weeks_ago
df['Stock Price (5wk ago),'] = five_weeks_ago
df['Stock Price (4wk ago),'] = four_weeks_ago
df['Stock Price (3wk ago),'] = three_weeks_ago
df['Stock Price (2wk ago),'] = two_weeks_ago
df['Stock Price (1wk ago),'] = one_week_ago
df['Stock Price (0wk ago),'] = yesterday
df['Sentiments'] = sentiments

cols = df.columns.tolist()
last = cols[-1:]
cols = cols[0:3] + last + cols[3:11]
df = df[cols]

file_name = "FinalProjectData_" + end_date_formatted + ".csv"
df.to_csv(file_name,index=False)

TSLA
[*********************100%***********************]  1 of 1 completed
Error on symbol TSLA for 2022-04-15
TM
[*********************100%***********************]  1 of 1 completed
Error on symbol TM for 2022-04-15
VOW3.DE
[*********************100%***********************]  1 of 1 completed
Error on symbol VOW3.DE for 2022-04-15
002594.SZ
[*********************100%***********************]  1 of 1 completed
DAI.DE
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DAI.DE: No data found, symbol may be delisted
Error on symbol DAI.DE for 2022-03-11
Error on symbol DAI.DE for 2022-03-18
Error on symbol DAI.DE for 2022-03-25
Error on symbol DAI.DE for 2022-04-01
Error on symbol DAI.DE for 2022-04-08
Error on symbol DAI.DE for 2022-04-15
Error on symbol DAI.DE for 2022-04-22
GM
[*********************100%***********************]  1 of 1 completed
Error on symbol GM for 2022-04-15
F
[*********************100%***********************]  1 of 1 completed
Er

In [5]:
df = pd.read_csv(file_name)
df

,country,Name,Symbol,Sentiments,marketcap,"Stock Price (6wk ago),","Stock Price (5wk ago),","Stock Price (4wk ago),","Stock Price (3wk ago),","Stock Price (2wk ago),","Stock Price (1wk ago),","Stock Price (0wk ago),"
0,United States,Tesla,TSLA,['Tesla earnings blow away expectations with r...,911205269504,795.3499755859375,905.3900146484375,1010.6400146484375,1084.5899658203125,1025.489990234375,error,1005.0499877929688
1,Japan,Toyota,TM,['Hyperspeed EV recruitment efforts offer tran...,274258280448,158.27000427246094,171.6999969482422,178.42999267578125,180.3699951171875,170.77999877929688,error,169.52999877929688
2,Germany,Volkswagen,VOW3.DE,No data!,124395549741,143.6999969482422,153.9199981689453,151.17999267578125,157.5800018310547,153.0399932861328,error,152.5399932861328
3,China,BYD,002594.SZ,No data!,107779133682,229.85000610351562,234.5,235.30999755859375,235.9600067138672,243.97999572753906,236.1999969482422,235.39999389648438
4,Germany,Daimler,DAI.DE,No data!,85824249620,error,error,error,error,error,error,error
5,United States,General Motors,GM,"[""NY Auto Show's top trends: Electric vehicles...",73682640896,41.5099983215332,44.81999969482422,43.650001525878906,42.959999084472656,39.349998474121094,error,39.84000015258789
6,United States,Ford,F,['Tech companies gear up to report earnings ne...,71636033536,16.040000915527344,16.860000610351562,16.469999313354492,16.649999618530273,15.050000190734863,error,15.130000114440918
7,Germany,BMW,BMW.DE,No data!,66793934199,72.41999816894531,75.9000015258789,77.88999938964844,79.0,77.0,error,79.19999694824219
8,Netherlands,Stellantis,STLA,['Stellantis Strengthens Hydrogen Fuel Cell Ve...,58517344256,14.59000015258789,16.299999237060547,15.859999656677246,16.350000381469727,14.850000381469727,error,14.020000457763672
9,Japan,Honda,HMC,"['GM, Honda Are Joining the Race for the Next ...",51654283264,26.450000762939453,28.31999969482422,28.440000534057617,28.299999237060547,26.059999465942383,error,25.93000030517578
